# Choix du pas de temps

## Pas fixe

On commence par coder une fonction qui résout l'équation différentielle $\dot{x} = f(x)$ avec le schéma d'Euler explicite.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_euler_explicit(f, x0, dt, t0, tf):
  n = int((tf-t0)//dt)
  t = np.linspace(t0, tf, n)
  x = np.zeros(n)
  x[0] = x0
  for j in range(1, n):
    x[j] = x[j-1] + dt*f(x[j-1])
  return t, x

### Exemple avec $f : x \mapsto x$

In [ ]:
def exp_solver(solver, dt, t0, tf = 5, x0 = 1):
    for delta_t in dt :
        t, x = solver(lambda x : x, x0, delta_t, t0, tf)
        plt.plot(t, x, label = f'dt = {delta_t}')
    plt.plot(t, [np.exp(i) for i in t], label = 'exp')
    plt.legend()
    plt.show()

In [ ]:
exp_solver(solve_euler_explicit, [0.1, 0.01], 0, 5)

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = solve_euler_explicit(lambda x : x, 1, dt, 0, 5)
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/dt)}.')
print('On en déduit que le schéma est bien convergeant à l\'ordre 1 avec c_v = 400.')


On choisit maintenant un schéma à l'ordre 2 : la méthode de Heun.

In [ ]:
def heun(f, x0, dt, t0, tf):
    n = int((tf-t0)/dt)
    t = np.linspace(0, dt*(n-1), n)
    x = np.zeros(n)
    x[0] = x0
    for j in range(1, n):
        x[j] = x[j-1] + dt/2*( f(x[j-1]) + f(x[j-1] + dt*f(x[j-1])) )
    return t, x

In [ ]:
exp_solver(heun, [0.1, 0.01], 0, 5)

Ce schéma est plus précis que celui d'Euler explicite. 

In [ ]:
for dt in [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] :
    t, x = heun(lambda x : x, 1, dt, 0, 5)
    x_exp = [np.exp(i) for i in t]
    dif = np.abs(x-x_exp)
    print(f'Pour dt = {dt}, le max de la différence divisé par dt est d\'environ {round(np.max(dif)/(dt**2))}.')
print('On en déduit que le schéma de Heun est convergeant à l\'ordre 2 avec c_v = 125.')

## Adaptation du pas de temps

On a : 

$e^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,x(s))ds - x^{j+1} = x^j + \int_{t_j}^{t_{j+1}} f(s,x(s))ds - ( x^{j} + (t_{j+1} - t_{j})*f(t_j,x^j))$

Soit : $e^{j+1} = \int_{t_j}^{t_{j+1}} (f(s,x(s)) - f(t_j,x^j))ds$

Or si $f$ est $C^1$, on a : $\forall s \in [ t_j , t_{j+1} ],  f(s,x(s)) = f(t_j,x(t_j)) + f'(t_j,x(t_j))\times s + O(s^2)$

Donc, on a : $e^{j+1} = \int_{t_j}^{t_{j+1}} f'(t_j,x(t_j))\times s + O(s^2)ds = f'(t_j,x(t_j))\times \frac{\Delta t_j^2}{2} + O(\Delta t_j^3)$

Ainsi : $e^{j+1} = O(\Delta t_j^2)$.

On nous donne la fonction suivante qui code un schéma d'Euler explicite à pas variable : 

In [ ]:
def solve_ivp_euler_explicit_variable_step(f, t0, x0, t_f, dtmin = 1e-16, dtmax = 0.01, atol = 1e-6):
    dt = dtmax/10; # initial integration step
    ts, xs = [t0], [x0]  # storage variables
    t = t0
    ti = 0  # internal time keeping track of time since latest storage point : must remain below dtmax
    x = x0
    while ts[-1] < t_f:
        while ti < dtmax:
            t_next, ti_next, x_next = t + dt, ti + dt, x + dt * f(x)
            x_back = x_next - dt * f(x_next)
            ratio_abs_error = atol / (linalg.norm(x_back-x)/2)
            dt = 0.9 * dt * sqrt(ratio_abs_error)
            if dt < dtmin:
                raise ValueError("Time step below minimum")
            elif dt > dtmax/2:
                dt = dtmax/2
            t, ti, x = t_next, ti_next, x_next
        dt2DT = dtmax - ti # time left to dtmax
        t_next, ti_next, x_next = t + dt2DT, 0, x + dt2DT * f(x)
        ts = vstack([ts,t_next])
        xs = vstack([xs,x_next])
        t, ti, x = t_next, ti_next, x_next
    return (ts, xs.T)